In [1]:
import numpy as np
import pyphi

from qutip import *
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *


Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



In [2]:
%load_ext autoreload
%autoreload 2

# Define states and matrices

In [3]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

In [4]:
pyphi.config.MEASURE = 'ID'
pyphi.config.PARTITION_TYPE = 'ALL'
pyphi.config.PICK_SMALLEST_PURVIEW = False

In [5]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([-1.+0.j, -0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# 2 Qubits - CNOT

In [6]:
cnot()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

## Classical results

In [8]:
print(pyphi.config)

{ 'ACTUAL_CAUSATION_MEASURE': 'PMI',
  'ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS': False,
  'CACHE_POTENTIAL_PURVIEWS': True,
  'CACHE_REPERTOIRES': True,
  'CACHE_SIAS': False,
  'CACHING_BACKEND': 'fs',
  'CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA': False,
  'CUT_ONE_APPROXIMATION': False,
  'FS_CACHE_DIRECTORY': '__pyphi_cache__',
  'FS_CACHE_VERBOSITY': 0,
  'LOG_FILE': 'pyphi.log',
  'LOG_FILE_LEVEL': 'INFO',
  'LOG_STDOUT_LEVEL': 'WARNING',
  'MAXIMUM_CACHE_MEMORY_PERCENTAGE': 50,
  'MEASURE': 'ID',
  'MONGODB_CONFIG': { 'collection_name': 'cache',
                      'database_name': 'pyphi',
                      'host': 'localhost',
                      'port': 27017},
  'NUMBER_OF_CORES': -1,
  'PARALLEL_COMPLEX_EVALUATION': False,
  'PARALLEL_CONCEPT_EVALUATION': False,
  'PARALLEL_CUT_EVALUATION': True,
  'PARTITION_TYPE': 'ALL',
  'PICK_SMALLEST_PURVIEW': False,
  'PRECISION': 6,
  'PRINT_FRACTIONS': True,
  'PROGRESS_BARS': True,
  'REDIS_CACHE': False,
  'REDIS_CONFIG'

In [9]:
CNOT_tpm = cnot().data.toarray()
#CNOT_tpm = pyphi.convert.le2be_state_by_state(CNOT_tpm.real)

In [10]:
CNOT_tpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [11]:
# pyphi convention for state ordering is little endian
CNOT_tpm = pyphi.convert.be2le_state_by_state(CNOT_tpm.real)

In [12]:
CNOT_tpm = pyphi.convert.to_2dimensional(pyphi.convert.state_by_state2state_by_node(CNOT_tpm.real))

In [13]:
network = pyphi.Network(CNOT_tpm)
subsystem = pyphi.Subsystem(network, (1,0))

In [14]:
ces = pyphi.compute.ces(subsystem)

In [13]:
ces

════════════════════════════════════════════════════════════
            Cause-effect structure (2 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              Concept: Mechanism = [n0], φ = 1              
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1                   ││  φ = 1                   │  
  │  Purview = [n0]          ││  Purview = [n0]          │  
  │  MIP:                    ││  MIP:                    │  
  │    n0     ∅              ││    n0     ∅              │  
  │    ─── ✕ ───             ││    ─── ✕ ───             │  
  │     ∅    n0              ││     ∅    n0              │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────────┐       │  
  │    │ S    Pr(S)  │  

## Quantum results

In [13]:
qgate = cnot()

In [14]:
rho = tensor(rho_plus, rho_zero)

In [15]:
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.5 0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.5 0. ]
 [0.  0.  0.  0. ]]

In [16]:
rho_e = evolve(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]

In [17]:
result = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  2.0


In [18]:
result

[{'mech': (0, 1),
  'purview': (0, 1),
  'phi': 2.0,
  'mip': 0,1    ∅ 
  ─── ✕ ───
   ∅    0,1,
  'state': [array([-0.70710678-0.j,  0.        +0.j,  0.        +0.j, -0.70710678+0.j])]}]